In [1]:
#pip install seaborn

In [2]:
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sb
from sklearn.preprocessing import MinMaxScaler
import os

# Data Loading

In [3]:
df_1= pd.read_excel(r'C:\Projects\andrew\data\cl_sell_buy.xlsx')  # Loading of cushing file 
df_2=pd.read_excel(r'C:\Projects\andrew\data\diffs.xlsx')  # Loading of pricing data
df_3=pd.read_excel(r'C:\Projects\andrew\data\margins.xlsx')




In [4]:
df_1.head()

,date,time_spread,dod_change,action
0,NaT,-0.83807,NaN,NaN
1,2011-01-03,-0.88000,NaN,SELL
2,2011-01-04,-1.04000,-0.16,SELL
3,2011-01-05,-1.12000,-0.08,SELL
4,2011-01-06,-1.37000,-0.25,SELL


In [5]:
start_date = '2015-01-10'

In [6]:
mask = (df_1['date'] > start_date)
df_1 = df_1.loc[mask]
df_1.dropna(axis=0, inplace=True)
df_1.describe()

,time_spread,dod_change
count,1589.000000,1589.000000
mean,-0.426570,0.000447
std,0.797868,0.251549
min,-7.290000,-5.350000
25%,-0.610000,-0.030000
50%,-0.270000,0.000000
75%,-0.060000,0.030000
max,2.550000,5.200000


In [7]:
df_1.head()

,date,time_spread,dod_change,action
1015,2015-01-12,-0.69,-0.06,SELL
1016,2015-01-13,-0.62,0.07,BUY
1017,2015-01-14,-0.48,0.14,BUY
1018,2015-01-15,-0.48,0.00,BUY
1019,2015-01-16,-0.44,0.04,BUY


In [8]:
df_2.head()

,ObservationDate,MEH-WTI,MID,LLS
0,2015-10-01,0.25,-0.20,2.25
1,2015-10-02,-0.40,-0.20,1.80
2,2015-10-05,-0.45,-0.25,1.60
3,2015-10-06,0.10,-0.20,1.65
4,2015-10-07,0.00,-0.05,2.00


In [9]:
mask = (df_2['ObservationDate'] > start_date)
df_2 = df_2.loc[mask]
df_2.dropna(axis=0, inplace=True)
df_2.describe()

,MEH-WTI,MID,LLS
count,1384.000000,1384.000000,1384.000000
mean,2.620629,-1.459429,3.327673
std,2.305474,3.811103,2.706635
min,-6.000000,-17.450000,-9.000000
25%,1.200000,-1.000000,1.700000
50%,1.800000,-0.150000,2.300000
75%,3.912500,0.400000,4.800000
max,25.000000,20.000000,50.000000


In [10]:
df_2.head()

,ObservationDate,MEH-WTI,MID,LLS
0,2015-10-01,0.25,-0.20,2.25
1,2015-10-02,-0.40,-0.20,1.80
2,2015-10-05,-0.45,-0.25,1.60
3,2015-10-06,0.10,-0.20,1.65
4,2015-10-07,0.00,-0.05,2.00


In [11]:
df_3.head()

,DateCalculated,Contract,MEH | USGC | HSK,MEH | USGC | FCC,WTI | P2 | FCC,WTI | P2 | HSK
0,2013-01-22,2013-02-01,NaN,NaN,5.78,9.29
1,2013-04-22,2013-05-01,NaN,NaN,18.92,19.77
2,2013-05-21,2013-06-01,NaN,NaN,15.99,16.87
3,2013-07-22,2013-08-01,NaN,NaN,-0.67,0.96
4,2013-10-21,2013-11-01,NaN,NaN,1.48,4.42


In [12]:
mask = (df_3['DateCalculated'] > start_date)
df_3 = df_3.loc[mask]
df_3.dropna(axis=0, inplace=True)
df_3.describe()

,MEH | USGC | HSK,MEH | USGC | FCC,WTI | P2 | FCC,WTI | P2 | HSK
count,715.000000,715.000000,715.000000,715.000000
mean,2.555455,0.997818,3.782629,5.967385
std,1.995024,2.077284,3.588861,3.573026
min,-1.530000,-3.810000,-3.270000,-1.240000
25%,1.085000,-0.530000,1.275000,3.505000
50%,2.350000,0.820000,3.320000,5.710000
75%,3.840000,2.180000,6.380000,8.395000
max,16.540000,14.740000,57.230000,60.230000


In [13]:
df1_df2=pd.merge(
    df_1,
    df_2,
    left_on=['date'],
    right_on=['ObservationDate']
)

In [14]:
combined_df=pd.merge(
    df1_df2,
    df_3,
    left_on=['date'],
    right_on=['DateCalculated']
)


In [15]:
combined_df['year'] = pd.DatetimeIndex(combined_df['date']).year
combined_df['month'] = pd.DatetimeIndex(combined_df['date']).month
combined_df['day'] = pd.DatetimeIndex(combined_df['date']).day
combined_df['weekday'] = pd.DatetimeIndex(combined_df['date']).weekday   # Monday is 0 and Sunday is 6

In [16]:
combined_df=combined_df.drop(columns=['date','ObservationDate','DateCalculated', 'Contract'])

In [17]:
lags = range(1, 5)  # 50 lags

df_lag=combined_df.assign(**{'{} (t-{})'.format(col, t): combined_df[col].shift(t)
    for t in lags
    for col in combined_df
})

In [18]:
df_lag.head()

,time_spread,dod_change,action,MEH-WTI,MID,LLS,MEH | USGC | HSK,MEH | USGC | FCC,WTI | P2 | FCC,WTI | P2 | HSK,...,MID (t-4),LLS (t-4),MEH | USGC | HSK (t-4),MEH | USGC | FCC (t-4),WTI | P2 | FCC (t-4),WTI | P2 | HSK (t-4),year (t-4),month (t-4),day (t-4),weekday (t-4)
0,-1.07,0.26,BUY,-0.50,0.55,1.80,0.87,1.76,1.91,0.66,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,-1.91,0.20,BUY,2.00,0.20,2.75,0.54,0.85,1.33,1.85,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,-1.61,0.09,BUY,1.85,-0.50,2.35,-1.33,0.28,2.73,2.37,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,-1.00,-0.41,SELL,0.80,-0.60,1.85,1.28,2.00,3.47,3.23,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,-0.36,0.23,BUY,1.65,0.00,1.50,1.60,1.74,5.78,5.99,...,0.55,1.8,0.87,1.76,1.91,0.66,2015.0,12.0,21.0,0.0


In [ ]:
input_df=combined_df.drop(columns=['action','ObservationDate','DateCalculated', 'Contract'])